In [1]:
!pip install langchain_community
!pip install replicate

In [2]:
from langchain_community.llms import Replicate
from google.colab import userdata
from tabulate import tabulate
import os
import pandas as pd
import time
import numpy as np

# Mengatur API Token
api_token = None
try:
    api_token = userdata.get('api_token')
    os.environ['REPLICATE_API_TOKEN'] = api_token
except Exception as e:
    print("Gagal mengambil 'api_token' dari Google Colab Secrets. Pastikan Anda sudah mengaturnya.")
    api_token = "MISSING"

In [3]:
# Model setup
model_name = "ibm-granite/granite-3.3-8b-instruct"
llm = None
if api_token != "MISSING":
    try:
        llm = Replicate(
            model=model_name,
            replicate_api_token=api_token,
        )
        print(f"Model {model_name} berhasil di-setup.")
    except Exception as e:
        print(f"Gagal setup model Replicate: {e}")
else:
    print("Tidak bisa setup model, API token tidak ditemukan.")

Model ibm-granite/granite-3.3-8b-instruct berhasil di-setup.


## *Import Dataset*

In [4]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("rezkanorhafizah/analisis-sentimen-publik-danantara")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/rezkanorhafizah/analisis-sentimen-publik-danantara/versions/1


In [5]:
import os

Data = os.listdir(path)
print(Data)

['data_clean_danantara.csv']


In [6]:
# Construct the full file path
file_path = os.path.join(path, Data[0])

# Read the CSV file into a DataFrame
Dataset = pd.read_csv(file_path)
print(Dataset)

           Username                                             tweets
0      @longgonemac  Awkwakkw sepi banget ini danantara, lagi grada...
1         @est_iwul                 Parah sih.. Danantara versi lite..
2       @mang_Deddy  Duitnya dari Danantara ya semoga tdk bermasala...
3    @gerbang_utama  Danantara resmi diluncukan terbesar Ke-7 di Du...
4    @gerbang_utama  Pemerintah akan jalankan 21 Proyek stratgeis d...
..              ...                                                ...
122  @pilihankucing  Akun anak haram kayak kau masih berharap cuan ...
123     @harmonisme  Ini lebih parah dari Mulyono, bansos  pakai pi...
124      @joko_win1  .hlah mbel...food estate ..lah..sekolah rakyat...
125        @Ezvan77  Uang tersebut masuk ke Danantara, supaya bisa ...
126      @SiantoKoe     Ambil dari danantara aja, nanti dijadikan bumn

[127 rows x 2 columns]


In [7]:
df = pd.DataFrame()
try:
  df = pd.read_csv(file_path)
  print(f"\nDataset {Data} berhasil dimuat. Jumlah baris: {len(df)}")
  print(df.head(20))
except Exception as e:
  print(f"Error: gagal memuat file {file_path}. Pastikan file sudah di-upload. Error: {e}")


Dataset ['data_clean_danantara.csv'] berhasil dimuat. Jumlah baris: 127
            Username                                             tweets
0       @longgonemac  Awkwakkw sepi banget ini danantara, lagi grada...
1          @est_iwul                 Parah sih.. Danantara versi lite..
2        @mang_Deddy  Duitnya dari Danantara ya semoga tdk bermasala...
3     @gerbang_utama  Danantara resmi diluncukan terbesar Ke-7 di Du...
4     @gerbang_utama  Pemerintah akan jalankan 21 Proyek stratgeis d...
5    @Brahim_maldini            Nunggu giliran Mulyono dibui sekeluarga
6         @bisaimpor                                  kan ada danantara
7           @Wawanda  Kalau beneran kita wajib curiga apakah mungkin...
8         @notoghejo                                Dipinjami Danantara
9          @yhnuluul  Harapan rakyat : \nDuit Danantara kali ya \nMu...
10  @akademimem96790  Danantara OTW, masih berjuang di liga champion...
11        @rahelputr  Danantara bukan sekadar ide, tetapi langk

## Klasifikasi

In [8]:
def parse_sentiment_respone(response):
  response_str = str(response)
  try:
    for line in response_str.split('\n'):
      if '- Sentiment:' in line:
        sentiment = line.split(':', 1)[1].strip().lower()
        if sentiment in ['positif', 'negatif', 'netral']:
          return sentiment
    # Jika format tidak ditemukan
    if 'positif' in response_str.lower(): return 'positif'
    if 'negatif' in response_str.lower(): return 'negatif'
    if 'netral' in response_str.lower(): return 'netral'
    return "parse error"
  except Exception as e:
    return "parse error"

In [9]:
def classify_sentiment_with_granite(tweet_text):
    """
    Menggunakan IBM Granite untuk klasifikasi,
    menggunakan teknik 'formatted_prompt'
    """
    tweet_text = str(tweet_text)

    prompt = f"""
Classify this tweet's sentiment as 'positif', 'negatif', or 'netral'.
Gunakan format ini: - Sentiment: [sentimen]

Tweet: "{tweet_text}"
"""
    try:
        response = llm.invoke(prompt)
        print("Panggilan API sukses, menunggu 11 detik...")
        time.sleep(11)
        return parse_sentiment_respone(response)
    except Exception as e:
        print(f"API Error: {e}")
        time.sleep(11) # Beri jeda jika ada error
        return "api_error"

In [11]:
if llm is not None and not df.empty:
    # Mengatur jumlah sampel yang diinginkan
    sample_size = 20
    df_sample = df.sample(n=sample_size, random_state=42)
    print(f"Mengambil {sample_size} sampel acak dari {len(df)} total tweet.")

    print("\n--- TUGAS 1: KLASIFIKASI SENTIMEN (SAMPEL) ---")
    print(f"Memulai proses klasifikasi untuk {len(df_sample)} tweet...")
    start_time = time.time()

    hasil_sentimen = [] # List untuk menampung hasil

    # Menggunakan iterrows() untuk loop yang lebih terkontrol
    for index, row in df_sample.iterrows():
        print(f"Memproses tweet #{index}: {row['tweets'][:50]}...") # Tampilkan 50 karakter pertama

        # Memangil fungsi
        sentiment = classify_sentiment_with_granite(row['tweets'])
        hasil_sentimen.append(sentiment)

        print(f"-> Hasil: {sentiment}")

    # Masukkan hasil kembali ke DataFrame
    df_sample['ai_sentiment'] = hasil_sentimen

    end_time = time.time()
    print(f"Selesai! Waktu proses: {end_time - start_time:.2f} detik.")

    # Tampilkan hasilnya
    print("\n--- Hasil Klasifikasi ---")
    print(tabulate(df_sample[['tweets','ai_sentiment']], headers='keys', tablefmt='fancy_grid'))

    # --- Hasil & Insight (Analytical Result) ---
    print("\n--- INSIGHT 1: Distribusi Sentimen ---")

    sentiment_counts = df_sample['ai_sentiment'].value_counts()
    print(sentiment_counts)

    print("\nPersentase Distribusi:")
    print(df_sample['ai_sentiment'].value_counts(normalize=True) * 100)

    # Simpan hasil SAMPEL ke CSV
    df_sample.to_csv("hasil_sentimen_danantara_SAMPEL.csv", index=False)
    print("\nFile 'hasil_sentimen_danantara_SAMPEL.csv' telah disimpan.")

    # --- Tugas AI Kedua (Summarization) ---
    print("\n--- TUGAS 2: SUMMARIZATION TEMA ---")
    positive_tweets = df_sample[df_sample['ai_sentiment'] == 'positif']['tweets']
    negative_tweets = df_sample[df_sample['ai_sentiment'] == 'negatif']['tweets']

    if not positive_tweets.empty:
        print("\nMeringkas tema-tema POSITIF...")
        # Gabungkan semua tweet positif
        positive_text = "\n".join(positive_tweets)

        # Gunakan prompt summarization Anda
        summary_prompt = f"""
Summarize this list of tweets by focusing on key points and positive themes.
Tweets:
{positive_text}
"""
        try:
            # Tambahkan jeda 11 detik untuk rate limit API
            print("Memanggil API untuk ringkasan positif... (menunggu 11 dtk)")
            time.sleep(11)

            summary_response = llm.invoke(summary_prompt)
            print("--- Ringkasan Tema Positif ---")
            print(summary_response)
        except Exception as e:
            print(f"Gagal meringkas tema positif: {e}")

    if not negative_tweets.empty:
        print("\nMeringkas tema-tema NEGATIF...")
        # Gabungkan semua tweet negatif
        negative_text = "\n".join(negative_tweets)

        summary_prompt = f"""
Summarize this list of tweets by focusing on key points and negative themes (complaints).
Tweets:
{negative_text}
"""
        try:
            # Tambahkan jeda 11 detik untuk rate limit API
            print("Memanggil API untuk ringkasan negatif... (menunggu 11 dtk)")
            time.sleep(11)

            summary_response = llm.invoke(summary_prompt)
            print("--- Ringkasan Tema Negatif (Keluhan) ---")
            print(summary_response)
        except Exception as e:
            print(f"Gagal meringkas tema negatif: {e}")

else:
    print("\nProses AI tidak dijalankan karena model atau data tidak siap.")

Mengambil 20 sampel acak dari 127 total tweet.

--- TUGAS 1: KLASIFIKASI SENTIMEN (SAMPEL) ---
Memulai proses klasifikasi untuk 20 tweet...
Memproses tweet #26: Danantara hadir untuk mengoptimalkan pengelolaan a...
Panggilan API sukses, menunggu 11 detik...
-> Hasil: positif
Memproses tweet #111: Tp kan baru di up skrng ketika banyak penolakan Da...
Panggilan API sukses, menunggu 11 detik...
-> Hasil: negatif
Memproses tweet #81: integritas ini memang sangat diperlukan di dananta...
Panggilan API sukses, menunggu 11 detik...
-> Hasil: positif
Memproses tweet #55: Sesimpel lu liat orang2 yang ada di danantara rata...
Panggilan API sukses, menunggu 11 detik...
-> Hasil: netral
Memproses tweet #44: berapa ratus ribu pegawai honorer yang udah mengab...
Panggilan API sukses, menunggu 11 detik...
-> Hasil: negatif
Memproses tweet #96: Selama dikelola dengan bersih dan profesional, Dan...
Panggilan API sukses, menunggu 11 detik...
-> Hasil: positif
Memproses tweet #80: Kalo sistemnya jelas ka

In [12]:
nama_file = "hasil_sentimen_danantara_SAMPEL.csv"

try:
  df_from_csv = pd.read_csv(nama_file)

  # Menampilkan Dataframe
  print(f"---Menampilkan data dari file: {nama_file} ---")

  # Menampilkan data menggunakna tabulate
  # headers='keys' -> menggunakan nama kolom DataFrame sebagai header
  # tablefmt -> format tabel (bisa diganti dengan 'grid', 'fancy_grid', dll.)
  print(tabulate(df_from_csv, headers='keys', tablefmt='fancy_grid'))

except FileNotFoundError:
  print(f"Error: File {nama_file} tidak ditemukan.")
except Exception as e:
  print(f"Error: Gagal memuat file {nama_file}. Pastikan file sudah di-upload. Error: {e}")

---Menampilkan data dari file: hasil_sentimen_danantara_SAMPEL.csv ---
╒════╤══════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤════════════════╕
│    │ Username         │ tweets                                                                                                                                                                                                                                                                     │ ai_sentiment   │
╞════╪══════════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╪═══════════